In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df_train=pd.read_csv('../input/google-stock-price/Google_Stock_Price_Train.csv')
df_test=pd.read_csv('../input/google-stock-price/Google_Stock_Price_Test.csv')

In [ ]:
df_train.info()

In [ ]:
df_train['Volume'].min()

In [ ]:
df_test.info()

In [ ]:
df_test.head()

In [ ]:
df_train['Close']=df_train['Close'].str.replace(',','')
df_train['Volume']=df_train['Volume'].str.replace(',','')
df_test['Volume']=df_test['Volume'].str.replace(',','')

In [ ]:
df_train['Close']=pd.to_numeric(df_train['Close'])
df_train['Volume']=pd.to_numeric(df_train['Volume'])
df_test['Volume']=pd.to_numeric(df_test['Volume'])

In [ ]:
plt.plot(pd.to_datetime(df_train['Date']), df_train['Open'],C='blue')
plt.plot(pd.to_datetime(df_train['Date']), df_train['High'],C='red')

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler=MinMaxScaler()

In [ ]:
X_scaler=scaler.fit_transform(df_train[['Open','High', 'Low','Close','Volume']])

In [ ]:
X_scaler

In [ ]:
X_scaler.shape

In [ ]:
len(X_scaler)

In [ ]:
X=[]
for i in range(len(X_scaler)-60):
    X.append(X_scaler[i:i+61, :])
X=np.array(X)

In [ ]:
X.shape

In [ ]:
X_train,y_train=X[:,:60],X[:,-1:]

In [ ]:
X_train.shape, y_train.shape

In [ ]:
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.models import Sequential

In [ ]:
model=Sequential()
model.add(LSTM(100,return_sequences=True, input_shape=(60,5)))
model.add(LSTM(100,return_sequences=True))
model.add(LSTM(100,return_sequences=True))
model.add(LSTM(100,return_sequences=False))
model.add(Dense(5))

In [ ]:
model.summary()

In [ ]:
model.compile(loss='mean_squared_error')

In [ ]:
hist = model.fit(X_train,y_train, batch_size=1,epochs=50)

In [ ]:
column=['Open','High', 'Low','Close','Volume']

In [ ]:
x_test=pd.concat([df_train[column][-60:],df_test[column]],axis=0)

In [ ]:
x_test=scaler.transform(x_test)

In [ ]:
x=[]
for i in range(len(x_test)-60):
    x.append(x_test[i:i+60,:])
x=np.array(x)

In [ ]:
x.shape

In [ ]:
y_pre=model.predict(x)

In [ ]:
y_pre

In [ ]:
y=scaler.inverse_transform(y_pre)

In [ ]:
columns=['Open','High', 'Low','Close','Volume']

In [ ]:
y=pd.DataFrame(y,columns=columns)

In [ ]:
y

In [ ]:
plt.plot(df_test['Close'], c='blue')
plt.plot(y['Close'], c='red')